In [1]:
#
import requests
import json
import pandas as pd

url="https://data.ny.gov/resource/ca8h-8gjq.json?$limit=8000&$offset=0"
response = requests.get(url)
print(response)
data= response.text
parsed = json.loads(data)
print(json.dumps(parsed, indent=4))

<Response [200]>


In [2]:
import pymongo
from pymongo import MongoClient
import urllib.parse

client = MongoClient("mongodb://localhost:27017")


mydb=client["Crimes"]
mycol=mydb["Crime_Analysis"]

In [3]:
mycol.insert_many(parsed)
print("All the data has been inserted on mongodb")

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 6086c02fb8cf77812ab0d769, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>

In [ ]:
x = mycol.find()
  
for data in x:
    print(data)

In [ ]:
Data=parsed
print("dataset stored in variable Data")

In [ ]:
#convertion into dataframe
df = pd.DataFrame(Data)
df

In [ ]:
#check the NA values
df.isna().sum()

In [ ]:
#drop or delete all NA
df=df.dropna()
df

In [ ]:
#convertion into integer datatype
df['year'] = df['year'].astype(int)

# fetch or select data only between year 2017 and 2019
df=df.loc[(df['year'] >= 2017) & (df['year'] <= 2019)]

In [ ]:
##convertion into integer datatype
df['total_index_crimes'] = df['total_index_crimes'].astype(int)
df['violent'] = df['violent'].astype(int)
df['property'] = df['property'].astype(int)

In [ ]:
# adding two columns
sum_column = df["total_index_crimes"] + df["violent"]

In [ ]:
df["total_crimes"] = sum_column

In [ ]:
sum_columns = df["total_crimes"] + df["property"]
df["total_crime"] = sum_columns

In [ ]:
df

In [ ]:
# drop or delete the below columns as all the crimes are sum in single column
df.drop('murder', axis='columns', inplace=True)
df.drop('forcible_rape', axis='columns', inplace=True)
df.drop('robbery', axis='columns', inplace=True)
df.drop('aggravated_assault', axis='columns', inplace=True)
df.drop('burglary', axis='columns', inplace=True)
df.drop('larceny', axis='columns', inplace=True)
df.drop('motor_vehicle_theft', axis='columns', inplace=True)
df.drop('total_crimes', axis='columns', inplace=True)
df.drop('property', axis='columns', inplace=True)
df.drop('violent', axis='columns', inplace=True)
df.drop('total_index_crimes', axis='columns', inplace=True)


In [ ]:
#delete or drop region column as all records carry same value 
df.drop('region',axis='columns', inplace=True)
df

In [ ]:
# convertion of dataframe to csv

import csv 
df.to_csv("NY_Crimes.csv",index=False)

In [ ]:
#Postgres database
import psycopg2

try:
    dbConnection = psycopg2.connect(user = "dap",
                                    password = "dap",
                                    host = "192.168.56.30",
                                    port = "5432",
                                    database = "postgres")
    dbConnection.set_isolation_level(0)# AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE dapgroupm;')
    dbCursor.close()
except (Exception, psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [ ]:
#create schema
createString = """
 CREATE TABLE NYCrime(
 county varchar(20),agency varchar,
 year integer, monthreported integer, 
 Id varchar, total_crimes integer
 );
 """

try:
    dbConnection = psycopg2.connect(
        user = "dap",password = "dap",
        host = "192.168.56.30",
        port = "5432",database = "dapgroupm")
    dbConnection.set_isolation_level(0)# AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception, psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): 
        dbConnection.close()

In [ ]:
#inserting values in schema or table
try:
    dbConnection = psycopg2.connect(
        user = "dap",password = "dap",
        host = "192.168.56.30",port = "5432",
        database = "dapgroupm")
    dbConnection.set_isolation_level(0)# AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    insertString = "INSERT INTO nycrime VALUES ('{}',"+"'{}',"+"'{}',"+"'{}',"+"'{}',"+"'{}')"
    with open('NY_Crimes.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)# skip the header
        for row in reader:
            dbCursor.execute(insertString.format(*row))
        dbConnection.commit()
        dbCursor.close()
except (Exception, psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [ ]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
sql = '''SELECT COUNT(total_crimes) as crime, County
FROM nycrime
GROUP BY County;
'''
try:
    dbConnection = psycopg2.connect(
        user = "dap",password = "dap",
        host = "192.168.56.30",port = "5432",
        database = "dapgroupm")
    nycrime_dataframe = sqlio.read_sql_query(sql, dbConnection)
except (Exception, psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): 
        dbConnection.close()

In [ ]:
nycrime_dataframe

In [4]:
type(nycrime_dataframe)

NameError: name 'nycrime_dataframe' is not defined